<a href="https://colab.research.google.com/github/ratral/hyd4gpv_py/blob/main/gpv_operation_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Loading the necessary libraries and functions.

In [13]:
#@title 0.1 Libreries/Packages

from pathlib import Path
from scipy.optimize import fsolve
# from google.colab import drive 
import numpy as np
import pandas as pd
import plotly.express as px


In [14]:
#@title 0.2 Vectorized Functions
"""
Created on Sun Sep 25 16:19:54 2022
@author: raul
"""

import numpy as np
from scipy.optimize import fsolve

def atm_pressure(masl) -> float:
  """Returns: Atmosphere  Pressure in [bar]"""
  return (1/1000)*((44331.514 - masl)/11880.516)**(1/0.1902632)
v_atm_pressure = np.vectorize(atm_pressure)

def vapor_pressure(temp_c) -> float:
  """Returns: Vapor pressure [bar]"""
  return (0.61121*np.exp((18.678-temp_c/234.5)*(temp_c/(257.14+temp_c))))/100
v_vapor_pressure = np.vectorize(vapor_pressure)

def density(temp_c) -> float:
  """Returns: density of water in [kg/m³]"""
  temp_k = temp_c + 273.15
  a = 0.14395; b = 0.0112; c = 649.727; d = 0.05107
  return  a/(b**(1 + (1-temp_k/c)**d))
v_density = np.vectorize(density)

def velocity(flow, diameter) -> float:
  """Returns: Velocity in [m/s]"""
  return (flow/3600)/((np.pi*diameter**2)/4)
v_velocity = np.vectorize(velocity)

def velocity_factor(flow, diameter) -> float:
  """Returns: Velocity factor in [bar]"""
  return (v_velocity(flow, diameter)**2 / (2*9.80665)) / 10
v_velocity_factor = np.vectorize(velocity_factor)

def absolute_pressure(gauge_pressure, masl) -> float:
  """Returns: Absolute Pressure in [bar]"""
  ap = v_atm_pressure(masl)
  return gauge_pressure + ap
v_absolute_pressure = np.vectorize(absolute_pressure)

def sigma_0(p_up, p_down, masl, temp_c) -> float:
  p1 = v_absolute_pressure(p_up, masl)
  p2 = v_absolute_pressure(p_down, masl)
  return (p1 - v_vapor_pressure(temp_c))/(p1-p2)
v_sigma_0 = np.vectorize(sigma_0)

def sigma_1(p_up, p_down, masl, temp_c) -> float:
  p1 = v_absolute_pressure(p_up, masl)
  p2 = v_absolute_pressure(p_down, masl)
  return (p2 - v_vapor_pressure(temp_c))/(p1-p2)
v_sigma_1 = np.vectorize(sigma_1)

def sigma_2(p_up, p_down, flow, diameter, masl, temp_c) -> float:
  p1 = v_absolute_pressure(p_up, masl)
  p2 = v_absolute_pressure(p_down, masl)
  v_factor = velocity_factor(flow, diameter)
  return (p2 - v_vapor_pressure(temp_c))/(p1 - p2 + v_factor)
v_sigma_2 = np.vectorize(sigma_2)

def flow_coefficent(p_up, p_down, flow, temp_c) -> float:
  ''' Return Flow Coefficent Kv in m3/h'''
  return flow*np.sqrt((v_density(temp_c)/1000)/(p_up - p_down))
v_flow_coefficent = np.vectorize(flow_coefficent)

def drop_coefficient(p_up, p_down, flow, diameter, temp_c) -> float:
  ''' Return Pressure Drop Coefficient - zeta'''
  kv = v_flow_coefficent(p_up, p_down, flow, temp_c)
  return (1/626.3)*((diameter*1000)**2/kv)**2
v_drop_coefficient = np.vectorize(drop_coefficient)

def kv_fun_zeta(diameter, zeta_value) -> float:
  ''' Return Flow Coefficent Kv in m3/h'''
  return ((diameter*1000)**2)/np.sqrt(626.3*zeta_value)
v_kv_fun_zeta = np.vectorize(kv_fun_zeta)

#-------------------------------------------------------------------------------
def resistance_coefficient(diameter, up_dn, down_dn) -> float:
  '''Return the Resistance coefficients of all fittings attached to 
  the control valve'''
  diameter *= 1000; up_dn *= 1000; down_dn *= 1000
  reducer  =  0.5 * ((1-(diameter/up_dn)**2)**2)
  diffuser =  ((1-(diameter/down_dn)**2)**2)
  bernulli =  (diameter/down_dn)**4 - (diameter/up_dn)**4
  return reducer + diffuser + bernulli

def piping_geometry_factor(f_coefficent, diameter, up_dn, down_dn) -> float:
  '''Return the piping geometry factor Fp'''
  diameter *= 1000; up_dn *= 1000; down_dn *= 1000
  rc = resistance_coefficient(diameter/1000, up_dn/1000, down_dn/1000)
  return (1 / np.sqrt(1+(rc*(f_coefficent/diameter**2)**2)/0.0016))
v_piping_geometry_factor = np.vectorize(piping_geometry_factor)


def combined_geometry_factor(f_coefficent, fl, diameter, up_dn, down_dn) -> float:
  '''
  Return the Combined liquid pressure recovery factor flp
  '''
  rc = resistance_coefficient(diameter/1000, up_dn/1000, down_dn/1000)
  return (fl / np.sqrt(1+(rc*(f_coefficent/diameter**2)**2)*(fl**2)/0.0016))
v_combined_geometry_factor = np.vectorize(combined_geometry_factor)

def critical_pressure_factor(temp_c) -> float:
  ''' ff is the Liquid critical pressure ratio factor'''
  pv = v_vapor_pressure(temp_c)
  #  the critical thermodynamic pressure for water is 221.2 bar
  pc = 221.2
  return 0.96-0.28*np.sqrt(pv/pc)
v_critical_pressure_factor = np.vectorize(critical_pressure_factor)

def max_differential_pressure(flp, fp, p1, temp_c) -> float:
  '''The maximum permissible differential pressure'''
  pv = v_vapor_pressure(temp_c)
  ff = v_critical_pressure_factor(temp_c)
  return ((flp/fp)**2)*(p1-ff*pv) 
v_permissible_differential_pressure = np.vectorize(max_differential_pressure)


#-------------------------------------------------------------------------------

# Plot functions for the Kv/Kvs
def drm_ll3(openinig,b,d,e):
  return d/(1+np.exp(b*(np.log(openinig)-np.log(e))))

# Plot functions for the Liquid pressure recovery factor Fl
def pressure_recovery_factor(openinig, fls, b, d, e) -> float:
  '''
  fl The liquid pressure recovery factor, fl, predicts the amount of pressure 
  '''
  sigma_value = 1/(fls**2) - 1
  kv_kvs = drm_ll3(openinig, b, d, e)
  return np.sqrt(1/(sigma_value * kv_kvs + 1))
v_pressure_recovery_factor = np.vectorize(pressure_recovery_factor)

# Incipient Cavitation
def incipient_cavitation(openinig, fls, b, d, e) -> float:
  xfz = 0.71
  kv_kvs = drm_ll3(openinig, b, d, e)
  return (1/(xfz * fls**2) - 1) * kv_kvs

# Solve kv_kvs function
def root_drm_ll3(kv_kvs,b,d,e):
  def fun(x,kv_kvs,b,d,e):
    return d/(1+np.exp(b*(np.log(x)-np.log(e))))-kv_kvs
  root = fsolve(fun, 50, args=(kv_kvs,b,d,e))
  return root
v_root_drm_ll3 = np.vectorize(root_drm_ll3)

# 1. The calculation for a group of operation data

In [15]:
#@title 1.1 Project Operation data Input
  
# Project Operation data
diameter = 0.200    # meter
up_dn    = 0.200    # meter
down_dn  = 0.200    # meter
masl = 1780         # meter
temp_c = 15         # Celsius
safety_factor = 1.3 # The factor of safety for the Kv, this must be >= 1 

# Operation Data:
#   {'p_up': 'bar', 'p_down': 'bar', 'flow': 'm3/h'}
operation_data = np.array(
    [["VRF-15.max",  4.439,	2.615,	396.0],
     ["VRF-16.max",	18.263,	5.241,	396.0],
     ["VRF-18.max",	 9.455,	2.615,	396.0],
     ["VRF-23.max",	20.215,	2.190,	396.0],
     ["VRF-24.max",	19.903,	2.190,	396.0],
     ["VRF-25.max",	19.132,	2.190,	396.0],
     ["VRF-26.max",	18.534,	2.347,	396.0],
     ["VRF-27.max",	17.439,	2.347,	396.0],
     ["VRF-28.max",	17.148,	2.347,	396.0],
     ["VRF-34.max",	18.314,	2.718,	396.0],
     ["VRF-15.min",	 4.368,	2.249,	252.0],
     ["VRF-16.min",	18.193,	5.077,	252.0],
     ["VRF-18.min",	 9.337,	2.249,	252.0],
     ["VRF-23.min",	20.185,	2.081,	252.0],
     ["VRF-24.min",	19.873,	2.081,	252.0],
     ["VRF-25.min",	19.100,	2.081,	252.0],
     ["VRF-26.min",	18.470,	2.148,	252.0],
     ["VRF-27.min",	17.372,	2.148,	252.0],
     ["VRF-28.min",	17.081,	2.148,	252.0],
     ["VRF-34.min",	18.299,	2.291,	252.0]]
  )

# Converting the operation data in a Dataframe
df2 = pd. \
  DataFrame(operation_data, columns = ['condition', 'p_up', 'p_down', 'flow']). \
  astype({ 'condition':'U16', 'p_up':'float32',	'p_down':'float32', 'flow':'float32'})
  
df2

,condition,p_up,p_down,flow
0,VRF-15.max,4.439000,2.615,396.0
1,VRF-16.max,18.263000,5.241,396.0
2,VRF-18.max,9.455000,2.615,396.0
3,VRF-23.max,20.215000,2.190,396.0
4,VRF-24.max,19.903000,2.190,396.0
5,VRF-25.max,19.132000,2.190,396.0
6,VRF-26.max,18.534000,2.347,396.0
7,VRF-27.max,17.438999,2.347,396.0
8,VRF-28.max,17.148001,2.347,396.0
9,VRF-34.max,18.313999,2.718,396.0


In [19]:
#@title 1.2 Operation Data Processingt

# pandas.DataFrame.set_index
df2.set_index('condition')

df2['p1']       = v_absolute_pressure(df2.p_up, masl)
df2['p2']       = v_absolute_pressure(df2.p_down, masl)
df2['dp']       = (df2.p_up - df2.p_down)
df2['velocity'] = v_velocity(df2.flow, diameter)
df2['v_factor'] = velocity_factor(df2.flow, diameter)
df2['sigma_0']  = v_sigma_0(df2.p_up, df2.p_down, masl, temp_c)
df2['sigma_1']  = v_sigma_1(df2.p_up, df2.p_down, masl, temp_c)
df2['sigma_2']  = sigma_2(df2.p_up, df2.p_down, df2.flow, diameter, masl, temp_c)
df2['kv']       = v_flow_coefficent(df2.p_up, df2.p_down, df2.flow, temp_c)
df2['zeta']     = v_drop_coefficient(df2.p_up, df2.p_down, df2.flow, diameter, temp_c)

# Calculating the maximum operating flow (kv) rate plus the safety factor. 
kvs_min = df2['kv'].max() * safety_factor

df2

,condition,p_up,p_down,flow,p1,p2,dp,velocity,v_factor,sigma_0,sigma_1,sigma_2,kv,zeta
0,VRF-15.max,4.439000,2.615,396.0,5.255911,3.431911,1.824000,3.501409,0.062508,2.872182,1.872182,1.810148,293.973887,29.561064
1,VRF-16.max,18.263000,5.241,396.0,19.079912,6.057911,13.022000,3.501409,0.062508,1.463896,0.463896,0.461680,110.022709,211.043947
2,VRF-18.max,9.455000,2.615,396.0,10.271911,3.431911,6.840000,3.501409,0.062508,1.499248,0.499248,0.494727,151.807468,110.853982
3,VRF-23.max,20.215000,2.190,396.0,21.031911,3.006911,18.025000,3.501409,0.062508,1.165873,0.165873,0.165300,93.515489,292.126177
4,VRF-24.max,19.903000,2.190,396.0,20.719911,3.006911,17.712999,3.501409,0.062508,1.168795,0.168795,0.168201,94.335494,287.069675
5,VRF-25.max,19.132000,2.190,396.0,19.948911,3.006911,16.941999,3.501409,0.062508,1.176476,0.176476,0.175827,96.458132,274.574294
6,VRF-26.max,18.534000,2.347,396.0,19.350912,3.163911,16.187000,3.501409,0.062508,1.194407,0.194407,0.193659,98.682012,262.338228
7,VRF-27.max,17.438999,2.347,396.0,18.255910,3.163911,15.091999,3.501409,0.062508,1.208512,0.208512,0.207652,102.199271,244.591847
8,VRF-28.max,17.148001,2.347,396.0,17.964912,3.163911,14.801001,3.501409,0.062508,1.212611,0.212611,0.211717,103.199037,239.875715
9,VRF-34.max,18.313999,2.718,396.0,19.130910,3.534911,15.595999,3.501409,0.062508,1.225562,0.225562,0.224661,100.534375,252.760034


In [ ]:
#@title 1.3 Reading and preselection of the valves

# Read the parameter of the VAG Valves.

# Select the valves that meet the maximum operating flow rate

# Plotting the functions and operation points.

In [8]:
#@title Plot $K_/K_{vs}$ Function
def main() -> None:

  # https://matplotlib.org/stable/index.html
  # https://matplotlib.org/stable/tutorials/text/mathtext.html
  # Read csv with de cylinder factors
  
  vale_types = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/gpv_data.csv')
  cols = ["brand",	"valve_name",	"cyl_name",	"dn_min",	 
          "dn_max",	"kv_b",	"kv_d",	"kv_e",	"zvs",	"fls"]

  vale_types = vale_types[cols] \
    .query('(brand == "VAG") & (cyl_name == "E")')

  xlist = np.linspace(1, 100, num = 99)

  ylist = drm_ll3(xlist, vale_types.kv_b[0], 
                  vale_types.kv_d[0],	vale_types.kv_e[0] )

  fig = px.line(x = xlist, y = ylist, 
                color_discrete_sequence = ['navy'])

  fig.update_layout(
      title = r'$K_{v}/K_{vs} \text{ Value}$',
     xaxis_title = r'Opening Degree [%]',
     yaxis_title = r'$Kv/Kvs \; [\%]$'
  )

  # https://plotly.com/python/axes/
  fig.update_yaxes(nticks=20)
  fig.update_xaxes(nticks=20)

  fig.show()

if __name__ == '__main__':
    main()